# Repurposhing Antminer S9 Control Board as FPGA Developmetn Board
- SoC ZYNQ 7010 (Dual Core ARM Cortex-A9 + FPGA Artix-7 28.000 Logic Cells) <br><br>
<img src="resource/Antminer S9.jpg" width="900px">


### 1. Prerequisites
- Antminer S9 Control Board
- SD Card flashed with PYNQ 2.5
    - Can be downloaded from : [https://github.com/cnchens/s9-v1.2-pynq2.5/tree/master](https://github.com/cnchens/s9-v1.2-pynq2.5/tree/master)
    - Flash using Balena Etcher
- Install `AntminerGPIO` Overlay from [https://github.com/guannan-he/Antminer_s9_pynq/tree/main](https://github.com/guannan-he/Antminer_s9_pynq/tree/main)

### 2. Load Overlay

In [1]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time

In [2]:
gpio = Overlay('antminerGPIO.bit')

- Hardware design for 'antminerGPIO.bit'<br><br>
<img src="https://raw.githubusercontent.com/guannan-he/Antminer_s9_pynq/main/antminerGPIO/pics/block_design_base.png" width="700px">

### 3. Play with On-board LED
- Using Xilinx AXI GPIO 
    - ip block `"plLED"`

In [3]:
led_instance = gpio.ip_dict["plLED"]
leds = AxiGPIO(led_instance).channel1

In [7]:
leds[0:4].write(0b1110)  # first led enlightened(near eth port)

In [9]:
leds[0:4].write(0b0111)  # last led enlightened

In [ ]:
# running 4 led

i = 0
while i < 100 :
    j = i % 8
    if j < 4:
        value = 0b1111 >> j  # This shifts the bits to the right
    else:
        value = 0b1111 >> (7 - j) # This reverses the pattern after 0000
    leds[0:4].write(value) 
    time.sleep(0.125)
    i += 1

In [12]:
leds[0:4].off()  # turn on all leds

In [13]:
leds[0:4].on()  # shut down all leds

<br><br><br><br>
### 4. Playing with GPIO in Port 0 - 9
- Using Xilinx AXI GPIO 
    - ip block `"plMiner0"` - `"plMiner9"`
<br><br>
<img src="resource/Antminer S9 Pin Mapping.png" width="1000px">

In [15]:
miner_instance = gpio.ip_dict["plMiner0"]
miner0 = AxiGPIO(miner_instance).channel1

- `miner0[0:4].write()` will `write` digital state into `pin 3`, `pin 2`, `pin 1`, `pin 0` on `"plMiner0"`
- `miner0[0].write()` will `write` digital state into `pin 3` on `"plMiner0"`
- `miner0[1].write()` will `write` digital state into `pin 2` on `"plMiner0"`
- `miner0[2].write()` will `write` digital state into `pin 1` on `"plMiner0"`
- `miner0[3].write()` will `write` digital state into `pin 0` on `"plMiner0"`

In [23]:
miner0[0:4].write(0b1111) # turn all miner0 output to HIGH

In [28]:
miner0[0:4].write(0b0000) # turn all miner0 output to LOW

<br><br><br><br>
### 5. Playing with 7 Segment
- Wiring diagram <br>
<img src="resource/7Segment Wiring.png" width="700px">

In [24]:
gpio_7segment_a_instant = gpio.ip_dict["plMiner1"]
gpio_7segment_a = AxiGPIO(gpio_7segment_a_instant).channel1

gpio_7segment_b_instant = gpio.ip_dict["plMiner2"]
gpio_7segment_b = AxiGPIO(gpio_7segment_b_instant).channel1

In [25]:
# turn all 7 segment element to ON
gpio_7segment_a[0:4].write(0b1111) # d c b a
gpio_7segment_b[0:4].write(0b1111) # dp h g f

In [ ]:
# 7 segmnet counting up and down

seven_segment_config = {
    0: (0b1111, 0b0011),  # Display '0'
    1: (0b0110, 0b0000),  # Display '1'
    2: (0b1011, 0b0101),  # Display '2'
    3: (0b1111, 0b0100),  # Display '3'
    4: (0b0110, 0b0110),  # Display '4'
    5: (0b1101, 0b0110),  # Display '5'
    6: (0b1101, 0b0111),  # Display '6'
    7: (0b0111, 0b0000),  # Display '7'
    8: (0b1111, 0b0111),  # Display '8'
    9: (0b1111, 0b0110),  # Display '9'
}

iterator = None
for j in range(10000) :
    iterator = range(9, -1, -1) if j % 2 else range(1, 9)
    
    for i in iterator :
        value = seven_segment_config[i]
        gpio_7segment_a[0:4].write(value[0])
        gpio_7segment_b[0:4].write(value[1])

        time.sleep(0.25)

In [27]:
# turn all 7 segment element to OFF
gpio_7segment_a[0:4].write(0b0000) 
gpio_7segment_b[0:4].write(0b0000) 

<br><br><br><br>
### 6. Playing with 7 Segment